## Consent API Demo
#### Responsible party - organization owning/stewarding the patient data
#### Responsible persona - Healthcare IT/Admin team

### Prerequisites

In [ ]:
PROJECT_ID="<>"
LOCATION="us-central1"
DATASET="<>"
CONSENT_BASE_URL='https://healthcare.googleapis.com/v1'
CONSENTSTORE="<>"
GCS_BUCKET="<>"

### Re-usable functions

In [ ]:
import json
def run_request(request):
    response = request.execute()
    print(json.dumps(
      response,
      sort_keys=True,
      indent=2
    ))
    return response
def get_session():
    """Creates an authorized Requests Session."""

    # Pass in the credentials and project ID. If none supplied, get them
    # from the environment.
    from google.oauth2 import service_account

    SCOPES = ['https://www.googleapis.com/auth/cloud-platform']
    SERVICE_ACCOUNT_FILE = "<json key file>"

    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    import googleapiclient.discovery

    service = googleapiclient.discovery.build('healthcare', 'v1beta1', credentials=credentials)
    
    parent = "projects/{}/locations/{}/datasets/{}".format(
        PROJECT_ID, LOCATION, DATASET
    )
    consent_parent = "projects/{}/locations/{}/datasets/{}/consentStores/{}".format(
        PROJECT_ID, LOCATION, DATASET,CONSENTSTORE
    )
    return service, parent, consent_parent

### Setup session

In [ ]:
svc, parent, consent_parent = get_session()

In [ ]:
## PART A: Executed by Admin

### Creation of Healthcare Dataset (Reference: https://cloud.google.com/healthcare/docs/how-tos/consent-managing)

In [ ]:
p = "projects/{}/locations/{}".format(
        PROJECT_ID, LOCATION
    )
request = svc.projects().locations().datasets().create(
        parent=p, datasetId=DATASET)

run_request(request)

### Creation of Consent Store "CONSENTSTORE" (Reference: https://cloud.google.com/healthcare/docs/how-tos/consent-managing)

In [ ]:
request = svc.projects().locations().datasets().consentStores().create(
        parent=parent, consentStoreId=CONSENTSTORE)

run_request(request)

### Creation of Consent Attributes - Resource (Reference: https://cloud.google.com/healthcare/docs/how-tos/consent-policies)

In [ ]:
## Resource - data_identifiable
consent_attr = { 
  # Required. Possible values for the attribute. The number of allowed values must not exceed 100. An empty list is invalid. The list can only be expanded after creation.  
  "allowedValues": ["identifiable", "de-identified"],
  "category": "RESOURCE",   # Required. The category of the attribute. The value of this field cannot be changed after creation.
  # Default values of the attribute in consents. If no default values are specified, it defaults to an empty value.    
  "consentDefaultValues": [],
  "description": "Indentifiable data", # A description of the attribute.
  "name": "data_identifiable", # Resource name of the attribute definition, of the form `projects/{project_id}/locations/{location_id}/datasets/{dataset_id}/consentStores/{consent_store_id}/attributeDefinitions/{attribute_definition_id}`.
}

request = svc.projects().locations().datasets().consentStores().attributeDefinitions().create(
        parent=consent_parent, attributeDefinitionId="data_identifiable", body=consent_attr)

run_request(request)
## Resource - data_type
consent_attr = { 
  # Required. Possible values for the attribute. The number of allowed values must not exceed 100. An empty list is invalid. The list can only be expanded after creation.  
  "allowedValues": ["activity", "vitals", "mentalhealth", "medicalrecord"],
  "category": "RESOURCE",   # Required. The category of the attribute. The value of this field cannot be changed after creation.
  # Default values of the attribute in consents. If no default values are specified, it defaults to an empty value.
  "consentDefaultValues": [],
  "description": "Type of data", # A description of the attribute.
  "name": "data_type", # Resource name of the attribute definition, of the form `projects/{project_id}/locations/{location_id}/datasets/{dataset_id}/consentStores/{consent_store_id}/attributeDefinitions/{attribute_definition_id}`.
}

request = svc.projects().locations().datasets().consentStores().attributeDefinitions().create(
        parent=consent_parent, attributeDefinitionId="data_type", body=consent_attr)
run_request(request)


In [ ]:
### Creation of Consent Attributes - Request (Reference: https://cloud.google.com/healthcare/docs/how-tos/consent-policies)

In [ ]:
consent_attr = {
    "allowedValues": ["provider", "healthplan", "partners"],
    "category": "REQUEST",
    "consentDefaultValues": [],
    "description": "Requester Type",
    "name": "requester_type",
}

request = svc.projects().locations().datasets().consentStores().attributeDefinitions().create(
    parent=consent_parent, attributeDefinitionId="requester_type", body=consent_attr)
run_request(request)
print("-------------------------------------------------------------------------------------------")
## Resource - data type
consent_attr = {
    "allowedValues": ["coaching", "research", "discount"],
    "category": "REQUEST",
    "consentDefaultValues": [],
    "description": "Requester Purpose",
    "name": "purpose",
}

request = svc.projects().locations().datasets().consentStores().attributeDefinitions().create(
    parent=consent_parent, attributeDefinitionId="purpose", body=consent_attr)
run_request(request)


## PART B: Consent by user/patient

In [ ]:
# Set username
import json
username = "user1" #Creation of consent for user1

### Creation of Consent - Artifact (Reference: https://googleapis.github.io/google-api-python-client/docs/dyn/healthcare_v1beta1.projects.locations.datasets.consentStores.consentArtifacts.html#create)

In [ ]:
data = {}
data['user_id'] = username
data['user_signature'] = {} 
data['user_signature']['user_id'] = username
data['consent_content_version'] = 'v1'
data['metadata'] = {} 
data['metadata']['client'] = 'mobile'

# Documentation

print(json.dumps(data, indent = 4))
    
request = svc.projects().locations().datasets().consentStores().consentArtifacts().create(
    parent=consent_parent,body=data)

response = run_request(request)
print(response)
artifact = response["name"]


### Creation of UserDataMapping (Reference: https://googleapis.github.io/google-api-python-client/docs/dyn/healthcare_v1beta1.projects.locations.datasets.consentStores.consentArtifacts.html#create)

In [ ]:
for a in ["activity", "vitals", "mentalhealth", "medicalrecord"]:
    # Create the userDataMapping
    # Use opaque id and expect another application to take care of transalting the opaqueid to physical resource location
    # the format of the opaque id is "useremail,"noun(activity/Vitals),physresource(bq/csv/fhir),id(table/csv
    # Create the payload
    # Create 2 version per data type
    ## Deidentified Data ID
    data = {}
    attr1 = {}
    attr2 = {}
    data["resource_attributes"] = []

    data["user_id"] = username
    data["data_id"] = username +  "," + a + "," + "deid"
    attr1["attribute_definition_id"] = "data_identifiable"
    attr1["values"] = ["de-identified"]
    data["resource_attributes"].append(attr1)

    attr2["attribute_definition_id"] = "data_type"
    attr2["values"] = [a]
    data["resource_attributes"].append(attr2)
    
    print("Creating user data mapping for user {} datatype {} and de-id".format(username,a))
    request = svc.projects().locations().datasets().consentStores().userDataMappings().create(
           parent=consent_parent,body=data)
    run_request(request)


    ## Identifiable Data ID
    data = {}
    attr1 = {}
    attr2 = {}
    data["user_id"] = username
    data["data_id"] = username +  "," + a + "," + "id"
    data["resource_attributes"] = []
    attr1["attribute_definition_id"] = "data_identifiable"
    attr1["values"] = ["identifiable"]
    data["resource_attributes"].append(attr1)

    attr2["attribute_definition_id"] = "data_type"
    attr2["values"] = [a]
    data["resource_attributes"].append(attr2)
    print("Creating user data mapping for datatype {} and id".format(a))
    request = svc.projects().locations().datasets().consentStores().userDataMappings().create( 
         parent=consent_parent,body=data)
    run_request(request)


In [ ]:
### Placeholder : Create the logical to physical mappings using a method of choice. eg. logical id {data_id": "testuser1,medicalrecord,deid"} needs to be understood at the time of policy enforcement.

### Creation of Consent itself (Reference: https://googleapis.github.io/google-api-python-client/docs/dyn/healthcare_v1beta1.projects.locations.datasets.consentStores.consents.html#create)

In [ ]:
# Sample consent data 
userconsentdata = ({})
userconsentdata["activity"] = {}
userconsentdata["vitals"] = {}
userconsentdata["medicalrecord"] = {}
userconsentdata["mentalhealth"] = {}
userconsentdata["de-identified"] = {}
userconsentdata["identifiable"] = {}

# Consent Choices
userconsentdata["activity"]["provider"] = True
userconsentdata["activity"]["healthplan"] = True
userconsentdata["activity"]["partners"] = True

userconsentdata["vitals"]["provider"] = True
userconsentdata["vitals"]["healthplan"] = True
userconsentdata["vitals"]["partners"] = True

userconsentdata["mentalhealth"]["provider"] = True
userconsentdata["mentalhealth"]["healthplan"] = True
userconsentdata["mentalhealth"]["partners"] = True

userconsentdata["medicalrecord"]["provider"] = True
userconsentdata["medicalrecord"]["healthplan"] = True
userconsentdata["medicalrecord"]["partners"] = True


userconsentdata["de-identified"]["research"] = True
userconsentdata["identifiable"]["discount"] = True
userconsentdata["identifiable"]["coaching"] = True

# Create consent itself
data = {}
data["user_id"] = username
data["policies"] = []
data["consent_artifact"]= artifact

for a in ["activity", "vitals", "mentalhealth", "medicalrecord"]:
    activityBy = []
    for b in ["provider", "healthplan", "partners"]:
        if userconsentdata[a][b] == True:
            activityBy.append(b)

    obj0 = {}
    obj0["resource_attributes"] = []
    obj = {}
    obj["attribute_definition_id"] = "data_type"
    obj["values"] = []
    obj["values"].append(a)
    obj0["resource_attributes"].append(obj)
    obj0["authorization_rule"] = {}
    obj0["authorization_rule"]["expression"] = "requester_type in {}".format(str(activityBy))
    data["policies"].append(obj0)

print(data)
print("------------------------")

print("Adding policies for identifiable data {}".format(username))

#Create the payload
accessBy = []
if (userconsentdata["identifiable"]["discount"] == True):
    accessBy.append("discount")
if (userconsentdata["identifiable"]["coaching"] == True):
    accessBy.append("coaching")

obj0 = {}
obj0['resource_attributes'] = []
obj = {}
obj['attribute_definition_id'] = 'data_identifiable'
obj['values'] = []
obj['values'].append('identifiable')
obj0['resource_attributes'].append(obj)
obj0['authorization_rule'] = {}
obj0['authorization_rule']['expression'] = 'purpose in {}'.format(str(accessBy))

data['policies'].append(obj0)
print(data)
print("------------------------")

print("Adding policies for de-identified  data  {}".format(username))
accessBy = []
if (userconsentdata["de-identified"]["research"] == True):
    accessBy.append("research")

obj0 = {}
obj0['resource_attributes'] = []
obj = {}
obj['attribute_definition_id'] = 'data_identifiable'
obj['values'] = []
obj['values'].append('de-identified')
obj0['resource_attributes'].append(obj)
obj0['authorization_rule'] = {}
obj0['authorization_rule']['expression'] = 'purpose in {}'.format(str(accessBy))

data['policies'].append(obj0)
print(data)
print("------------------------")

request = svc.projects().locations().datasets().consentStores().consents().create(
    parent=consent_parent,body=data)

run_request(request)


## PART C: Policy Enforcement

### Querying the Consent Store (Reference: https://googleapis.github.io/google-api-python-client/docs/dyn/healthcare_v1beta1.projects.locations.datasets.consentStores.html#queryAccessibleData)

In [ ]:
# Sample request parameters

#who = "partners"
#purpose = "research"
#datatype = "mentalhealth"
#identifiable = False
#if (datatype != "any"):
#    data['resource_attributes']['data_type'] = datatype
#data['request_attributes']['requester_type'] = who
who = "partners"
purpose = "research"
datatype = "medicalrecord"
identifiable = "de-identified"

data = {}
data['gcs_destination'] = {}
data['gcs_destination']['uri_prefix'] =  GCS_BUCKET
data['resource_attributes'] = {}
data['request_attributes'] = {}

if (identifiable != "any"):
    data['resource_attributes']['data_identifiable'] = identifiable
if (purpose != "any"):
    data['request_attributes']['purpose'] = purpose




print(data)
request = svc.projects().locations().datasets().consentStores().queryAccessibleData(
    consentStore = consent_parent, body=data)
run_request(request)
print ("------------------------------------------")


In [ ]:
!gsutil ls -l <GCS_BUCKET>

In [ ]:
!gsutil cat <GCS_FILENAME>

In [ ]:
# Sample request parameters

#who = "partners"
#purpose = "research"
#datatype = "mentalhealth"
#identifiable = False
#if (datatype != "any"):
#    data['resource_attributes']['data_type'] = datatype
#data['request_attributes']['requester_type'] = who
who = "partners"
purpose = "research"
datatype = "medicalrecord"
identifiable = "de-identified"

data = {}
data['gcs_destination'] = {}
data['gcs_destination']['uri_prefix'] = GCS_BUCKET
data['resource_attributes'] = {}
data['request_attributes'] = {}

data['resource_attributes']['data_type'] = "medicalrecord"
data['request_attributes']['requester_type'] = "partners"

print(data)
request = svc.projects().locations().datasets().consentStores().queryAccessibleData(
    consentStore = consent_parent, body=data)
run_request(request)
print ("------------------------------------------")


In [ ]:
!gsutil ls -l <GCS_BUCKET>

In [ ]:
!gsutil cat <GCS_FILENAME>